In [106]:
# %pip install biopython filelock

In [105]:
# %pip install idtrack

In [2]:
import scanpy as sc
import pandas as pd
import numpy as np
import warnings
import anndata as ad
warnings.filterwarnings("ignore")
from scipy.sparse import csr_matrix
from IPython.display import display, HTML

In [2]:
adata_sc = sc.read_h5ad('../single_cell_int/adata_sc_int_outlier.h5ad')
# adata_sn = sc.read_h5ad('../single_nuc_int/adata_nuc_int_outlier.h5ad')

In [3]:
import pickle
with open('kemal_map_result.pickle', 'rb') as f:
    results_dict = pickle.load(f)

pd.DataFrame.from_dict(results_dict['binned_conversions_adata_sc_genes'], orient='index').T

df = pd.DataFrame(results_dict['matching_adata_sc_genes']) #.groupby('final_database').size()

df_subset = df[['last_node', 'query_id']]

# Assuming df_subset['last_node'] contains a list of tuples
df_subset['col1'] = df_subset['last_node'].apply(lambda x: x[0][0] if len(x) > 0 else None)
df_subset['col2'] = df_subset['last_node'].apply(lambda x: x[0][1] if len(x) > 0 else None)

df_subset

from collections import defaultdict
ensembl_dict = defaultdict(set)
for index, row in df_subset.iterrows():
    ensembl_dict[row['col1']].add(row['query_id'])
    # ensembl_dict[row['col2']].add(row['query_id'])
filtered_dict = {key: list(value) for key, value in ensembl_dict.items() if len(value) > 1}

filtered_dict.pop(None)

mapped_df = pd.DataFrame.from_dict(filtered_dict, orient='index')

dict_map = dict(list(zip(df_subset.query_id, df_subset.col1)))

adata_sc.var['ensembl_id'] = adata_sc.var_names.map(dict_map)

len(adata_sc.var.ensembl_id.unique())

var_names = pd.Series(adata_sc.var.ensembl_id)
unique_ensembl_ids = var_names.unique()

# new_X = csr_matrix((adata_sc.X.shape[0], len(unique_ensembl_ids)))

# new_X

<617160x32197 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>

In [ ]:
# for i, ensembl_id in enumerate(unique_ensembl_ids):
#     indices = np.where(var_names == ensembl_id)[0]    
#     new_X[:, i] = adata_sc.X[:, indices].sum(axis=1)
#TODO remove if not needed

# look at the mapped df to see how many genes are mapped to one ensembl ID

In [9]:
mapped_df['None_count'] = mapped_df.isna().sum(axis=1)


In [10]:
mapped_df.sort_values('None_count')

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,None_count
ENSG00000270276.3,HIST1H4K,HIST1H4F,HIST1H4E,HIST1H4J,HIST2H4B,HIST1H4B,HIST1H4H,HIST1H4L,HIST1H4I,HIST2H4A,HIST1H4C,HIST4H4,HIST1H4D,HIST1H4A,0
ENSG00000197409.8,HIST1H3E,HIST1H3A,HIST1H3I,HIST1H3J,H3C4,HIST1H3F,HIST1H3C,HIST1H3G,HIST1H3B,HIST1H3D,HIST1H3H,None,None,None,3
ENSG00000228495.3,RP11-69I8.3,RP3-523C21.2,AL117378.1,LINC01013,AL133346.1,RP11-69I8.2,AL021408.1,None,None,None,None,None,None,None,7
ENSG00000285756.2,FAM239A,RP11-706O15.1,RP11-706O15.5,FAM239B,RP11-706O15.3,BX890604.2,BX890604.1,None,None,None,None,None,None,None,7
ENSG00000196787.4,HIST1H2AG,HIST1H2AK,HIST1H2AI,HIST1H2AL,HIST1H2AM,H2AC11,None,None,None,None,None,None,None,None,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000214189.10,ZNF788P,ZNF788,None,None,None,None,None,None,None,None,None,None,None,None,12
ENSG00000204623.11,ZNRD1-AS1,ZNRD1ASP,None,None,None,None,None,None,None,None,None,None,None,None,12
ENSG00000212643.3,ZRSR2P1,ZRSR1,None,None,None,None,None,None,None,None,None,None,None,None,12
ENSG00000153975.10,ZUP1,ZUFSP,None,None,None,None,None,None,None,None,None,None,None,None,12


In [11]:
unique_genes = len(set(mapped_df.iloc[:, :14].values.flatten()))
ensembl_ids = mapped_df.sort_values('None_count').shape[0]

# Format and display the text with larger font
display(HTML(f'<h3>{unique_genes} unique genes are mapped to {ensembl_ids} ensembl IDs</h3>'))

# Change adata_sc.var

In [12]:
adata_sc.var.ensembl_id = adata_sc.var.ensembl_id.astype(str)
# sc.get.aggregate(adata_sc, by='ensembl_id', func='sum', axis=1)
adata_sc.var.reset_index(inplace=True)
adata_sc.var_names = adata_sc.var.ensembl_id
adata_sc.var.rename(columns={'index':'gene_name'}, inplace=True)

# lots are mappeed to None, check for the reason why

In [18]:
no_hits = adata_sc.var[adata_sc.var_names == 'None'].sort_values(by='total_counts', ascending=False)

In [19]:
no_hits

,gene_name,n_cells,highly_variable,means,dispersions,dispersions_norm,highly_variable_nbatches,highly_variable_intersection,n_cells_by_counts,mean_counts,log1p_mean_counts,pct_dropout_by_counts,total_counts,log1p_total_counts,mito,ensembl_id
ensembl_id,,,,,,,,,,,,,,,,
None,MTRNR2L12,316076,True,1.249051,2.622231,0.775115,6,False,310687,2.808878,1.337335,49.658597,1733527.0,14.365669,False,None
None,MTRNR2L10,8835,False,0.211630,1.047025,0.194332,2,False,8801,0.044084,0.043140,98.573952,27207.0,10.211266,False,None
None,AL365361.1,15823,False,0.030942,0.586658,0.334883,4,False,14954,0.027565,0.027192,97.576965,17012.0,9.741733,False,None
None,RP11-343N15.5,10174,False,0.065022,0.498654,0.078535,0,False,10174,0.025894,0.025565,98.351481,15981.0,9.679218,False,None
None,RP11-427H3.3,11151,False,0.062667,0.504670,0.090422,0,False,11151,0.025527,0.025206,98.193175,15754.0,9.664913,False,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
None,LLNLR-299G3.1,5,False,0.000047,0.069827,-0.013157,0,False,5,0.000008,0.000008,99.999190,5.0,1.791759,False,None
None,CTC-451A6.4,5,False,0.000038,0.065715,-0.141537,0,False,5,0.000008,0.000008,99.999190,5.0,1.791759,False,None
None,PRR25,1505,False,0.000002,-0.112178,-0.261942,0,False,1,0.000002,0.000002,99.999838,1.0,0.693147,False,None


In [15]:
# take for example AC006273.7

In [100]:
adata_sc.var[adata_sc.var.gene_name.str.contains('AC006273.7')]

,gene_name,n_cells,highly_variable,means,dispersions,dispersions_norm,highly_variable_nbatches,highly_variable_intersection,n_cells_by_counts,mean_counts,log1p_mean_counts,pct_dropout_by_counts,total_counts,log1p_total_counts,mito,ensembl_id
ensembl_id,,,,,,,,,,,,,,,,
None,AC006273.7,136,False,0.000543,0.071288,-0.512172,0,False,136,0.000237,0.000237,99.977964,146.0,4.990433,False,NaN


In [101]:
#check in original df
df[df.query_id == 'AC006273.7']

,target_id,last_node,final_database,graph_id,query_id,no_corresponding,no_conversion,no_target
895,[],[],None,AC006273.7,AC006273.7,False,True,False


# google search for this gene
# turns out .7 is added for some reason, no hits with exact gene name on NCBI
# hit with AC006273
https://www.ncbi.nlm.nih.gov/gene/5725

# most get hits when '.number' is removed

In [114]:
df[df.query_id.str.contains('AC006273')]

,target_id,last_node,final_database,graph_id,query_id,no_corresponding,no_conversion,no_target
892,[ENSG00000272473.2],"[(ENSG00000272473.2, ENSG00000272473.2)]",ensembl_gene,AC006273.1,AC006273.1,False,False,True
893,[ENSG00000272473.2],"[(ENSG00000272473.2, ENSG00000272473.2)]",ensembl_gene,AC006273.4,AC006273.4,False,False,True
894,[LINC01836],"[(ENSG00000267530.5, LINC01836)]",HGNC Symbol,AC006273.5,AC006273.5,False,False,False
895,[],[],None,AC006273.7,AC006273.7,False,True,False


In [117]:
df[df.query_id.str.contains('MTRNR2')]

,target_id,last_node,final_database,graph_id,query_id,no_corresponding,no_conversion,no_target
24974,[JPT1],"[(ENSG00000189159.16, JPT1)]",HGNC Symbol,MTRNR2L1,MTRNR2L1,False,False,False
24975,[],[],None,MTRNR2L10,MTRNR2L10,False,True,False
24976,[],[],None,MTRNR2L11,MTRNR2L11,False,True,False
24977,[],[],None,MTRNR2L12,MTRNR2L12,False,True,False
24978,[MIR1973],"[(ENSG00000284253.1, MIR1973)]",HGNC Symbol,MTRNR2L13,MTRNR2L13,False,False,False
24979,[NOTCH2],"[(ENSG00000134250.21, NOTCH2)]",HGNC Symbol,MTRNR2L2,MTRNR2L2,False,False,False
24980,[PSMB4],"[(ENSG00000159377.11, PSMB4)]",HGNC Symbol,MTRNR2L3,MTRNR2L3,False,False,False
24981,[],[],None,MTRNR2L4,MTRNR2L4,False,True,False
24982,[],[],None,MTRNR2L5,MTRNR2L5,False,True,False
24983,[MTRNR2L6],"[(ENSG00000282017.1, MTRNR2L6)]",HGNC Symbol,MTRNR2L6,MTRNR2L6,False,False,False


In [16]:
# try the genes with no ensemble IDs to map with mygene

In [17]:
import mygene

In [24]:
mg = mygene.MyGeneInfo()

In [28]:
gene_name = no_hits['gene_name']

In [29]:
mg.query(gene_name, scopes='symbol', fields='ensembl.gene', species='human')

{'took': 3,
 'total': 1,
 'max_score': 36.440697,
 'hits': [{'_id': '101926957', '_score': 36.440697}]}

In [30]:
def get_ensembl_id(gene_name):
    try:
        result = mg.query(gene_name, scopes='symbol', fields='ensembl.gene', species='human')
        if 'hits' in result and len(result['hits']) > 0:
            return result['hits'][0]['ensembl']['gene'] if 'ensembl' in result['hits'][0] else None
    except:
        return None

# Apply the function to the 'gene_name' column to get Ensembl IDs
no_hits['ensembl_id'] = no_hits['gene_name'].apply(get_ensembl_id)

In [31]:
no_hits

,gene_name,n_cells,highly_variable,means,dispersions,dispersions_norm,highly_variable_nbatches,highly_variable_intersection,n_cells_by_counts,mean_counts,log1p_mean_counts,pct_dropout_by_counts,total_counts,log1p_total_counts,mito,ensembl_id
ensembl_id,,,,,,,,,,,,,,,,
None,MTRNR2L12,316076,True,1.249051,2.622231,0.775115,6,False,310687,2.808878,1.337335,49.658597,1733527.0,14.365669,False,None
None,MTRNR2L10,8835,False,0.211630,1.047025,0.194332,2,False,8801,0.044084,0.043140,98.573952,27207.0,10.211266,False,None
None,AL365361.1,15823,False,0.030942,0.586658,0.334883,4,False,14954,0.027565,0.027192,97.576965,17012.0,9.741733,False,None
None,RP11-343N15.5,10174,False,0.065022,0.498654,0.078535,0,False,10174,0.025894,0.025565,98.351481,15981.0,9.679218,False,None
None,RP11-427H3.3,11151,False,0.062667,0.504670,0.090422,0,False,11151,0.025527,0.025206,98.193175,15754.0,9.664913,False,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
None,LLNLR-299G3.1,5,False,0.000047,0.069827,-0.013157,0,False,5,0.000008,0.000008,99.999190,5.0,1.791759,False,None
None,CTC-451A6.4,5,False,0.000038,0.065715,-0.141537,0,False,5,0.000008,0.000008,99.999190,5.0,1.791759,False,None
None,PRR25,1505,False,0.000002,-0.112178,-0.261942,0,False,1,0.000002,0.000002,99.999838,1.0,0.693147,False,None


In [36]:
df_subset[df_subset.col1.astype(str) == 'None']

,last_node,query_id,col1,col2
134,[],AC000003.2,None,None
338,[],AC004257.3,None,None
345,[],AC004447.2,None,None
364,[],AC004520.1,None,None
415,[],AC004801.6,None,None
...,...,...,...,...
37152,[],U91328.20,None,None
37819,[],WI2-1959D15.1,None,None
37971,[],XXyac-YX155B6.6,None,None
38061,[],Z83851.2,None,None


In [40]:
import re

# Function to remove suffixes from gene names
def remove_suffix(gene_name):
    return re.sub(r'\.\d+$', '', gene_name)

# Apply the function to the 'query_id' column
no_hits['query_id_clean'] = no_hits['gene_name'].apply(remove_suffix)

In [42]:
no_hits.head()

,gene_name,n_cells,highly_variable,means,dispersions,dispersions_norm,highly_variable_nbatches,highly_variable_intersection,n_cells_by_counts,mean_counts,log1p_mean_counts,pct_dropout_by_counts,total_counts,log1p_total_counts,mito,ensembl_id,query_id_clean,ensembl_id_clean
ensembl_id,,,,,,,,,,,,,,,,,,
None,MTRNR2L12,316076,True,1.249051,2.622231,0.775115,6,False,310687,2.808878,1.337335,49.658597,1733527.0,14.365669,False,None,MTRNR2L12,NaN
None,MTRNR2L10,8835,False,0.211630,1.047025,0.194332,2,False,8801,0.044084,0.043140,98.573952,27207.0,10.211266,False,None,MTRNR2L10,NaN
None,AL365361.1,15823,False,0.030942,0.586658,0.334883,4,False,14954,0.027565,0.027192,97.576965,17012.0,9.741733,False,None,AL365361,NaN
None,RP11-343N15.5,10174,False,0.065022,0.498654,0.078535,0,False,10174,0.025894,0.025565,98.351481,15981.0,9.679218,False,None,RP11-343N15,NaN
None,RP11-427H3.3,11151,False,0.062667,0.504670,0.090422,0,False,11151,0.025527,0.025206,98.193175,15754.0,9.664913,False,None,RP11-427H3,NaN


In [43]:
# Re-query with cleaned gene names
gene_names_clean = no_hits['query_id_clean'].dropna().unique().tolist()
results_clean = mg.querymany(gene_names_clean, scopes='symbol', fields='ensembl.gene', species='human')

# Convert results to a dictionary for quick lookup
mapping_clean = {item['query']: item.get('ensembl', {}).get('gene') for item in results_clean if 'ensembl' in item}

# Map the results back to the DataFrame
no_hits['ensembl_id_clean'] = no_hits['query_id_clean'].map(mapping_clean)

378 input query terms found no hit:	['AL365361', 'RP11-343N15', 'RP11-427H3', 'AL592284', 'RP11-94L15', 'AL512274', 'RP11-145M9', 'RP11-


In [39]:
no_hits.ensembl_id_clean.unique()

array([nan])

In [45]:
results_alt = mg.querymany(no_hits['query_id_clean'], scopes=['alias', 'accession'], fields='ensembl.gene', species='human')

413 input query terms found no hit:	['MTRNR2L12', 'MTRNR2L10', 'AL365361', 'RP11-343N15', 'RP11-427H3', 'AL592284', 'RP11-94L15', 'AL512


In [48]:
import pandas as pd
import mygene

# Initialize the mygene query service
mg = mygene.MyGeneInfo()

# Clean up query IDs by removing suffixes
def remove_suffix(gene_name):
    return re.sub(r'\.\d+$', '', gene_name)

# no_hits['query_id_clean'] = no_hits['query_id'].apply(remove_suffix)

# Convert cleaned gene names to a list
gene_names_clean = no_hits['query_id_clean'].dropna().unique().tolist()

# Query HGNC using the cleaned gene names
results = mg.querymany(gene_names_clean, scopes='hgnc.symbol', fields='ensembl.gene', species='human')

# Convert results to a dictionary for quick lookup
mapping = {item['query']: item.get('ensembl', {}).get('gene') for item in results if 'ensembl' in item}

# Map the results back to the DataFrame
no_hits['ensembl_id'] = no_hits['query_id_clean'].map(mapping)


395 input query terms found no hit:	['MTRNR2L12', 'MTRNR2L10', 'AL365361', 'RP11-343N15', 'RP11-427H3', 'AL592284', 'RP11-94L15', 'AL512


In [49]:
no_hits.head()

,gene_name,n_cells,highly_variable,means,dispersions,dispersions_norm,highly_variable_nbatches,highly_variable_intersection,n_cells_by_counts,mean_counts,log1p_mean_counts,pct_dropout_by_counts,total_counts,log1p_total_counts,mito,ensembl_id,query_id_clean,ensembl_id_clean
ensembl_id,,,,,,,,,,,,,,,,,,
None,MTRNR2L12,316076,True,1.249051,2.622231,0.775115,6,False,310687,2.808878,1.337335,49.658597,1733527.0,14.365669,False,NaN,MTRNR2L12,NaN
None,MTRNR2L10,8835,False,0.211630,1.047025,0.194332,2,False,8801,0.044084,0.043140,98.573952,27207.0,10.211266,False,NaN,MTRNR2L10,NaN
None,AL365361.1,15823,False,0.030942,0.586658,0.334883,4,False,14954,0.027565,0.027192,97.576965,17012.0,9.741733,False,NaN,AL365361,NaN
None,RP11-343N15.5,10174,False,0.065022,0.498654,0.078535,0,False,10174,0.025894,0.025565,98.351481,15981.0,9.679218,False,NaN,RP11-343N15,NaN
None,RP11-427H3.3,11151,False,0.062667,0.504670,0.090422,0,False,11151,0.025527,0.025206,98.193175,15754.0,9.664913,False,NaN,RP11-427H3,NaN


In [50]:
pwd

'/ictstr01/home/aih/shrey.parikh/PDAC/PDAC_Final/Notebooks'

In [130]:
# load ensemble csv
ensemble_csv = pd.read_table('../result.txt', header=None, names=column_names)

In [128]:
attributes = """
<Attribute name = "ensembl_gene_id" />
<Attribute name = "ensembl_transcript_id" />
<Attribute name = "external_synonym" />
<Attribute name = "external_transcript_name" />
<Attribute name = "external_gene_name" />
<Attribute name = "entrezgene_id" />
<Attribute name = "hgnc_id" />
<Attribute name = "end_position" />
<Attribute name = "start_position" />
<Attribute name = "chromosome_name" />
<Attribute name = "ensembl_gene_id_version" />
<Attribute name = "ensembl_transcript_id_version" />
"""

column_names = re.findall(r'Attribute name = "(.*?)"', attributes)

In [131]:
ensemble_csv.head()

,ensembl_gene_id,ensembl_transcript_id,external_synonym,external_transcript_name,external_gene_name,entrezgene_id,hgnc_id,end_position,start_position,chromosome_name,ensembl_gene_id_version,ensembl_transcript_id_version
0,ENSG00000286112,ENST00000651925,CCBL1,KYAT1-211,KYAT1,883.0,NaN,128881931,128819663,9,ENSG00000286112.1,ENST00000651925.1
1,ENSG00000286112,ENST00000651925,CCBL1,KYAT1-211,KYAT1,127739546.0,NaN,128881931,128819663,9,ENSG00000286112.1,ENST00000651925.1
2,ENSG00000171097,ENST00000302586,CCBL1,KYAT1-201,KYAT1,883.0,HGNC:1564,128882494,128832942,9,ENSG00000171097.14,ENST00000302586.8
3,ENSG00000171097,ENST00000462722,CCBL1,KYAT1-207,KYAT1,883.0,HGNC:1564,128882494,128832942,9,ENSG00000171097.14,ENST00000462722.5
4,ENSG00000171097,ENST00000320665,CCBL1,KYAT1-202,KYAT1,883.0,HGNC:1564,128882494,128832942,9,ENSG00000171097.14,ENST00000320665.10


In [101]:
' '.join(no_hits['gene_name'].tolist())

'MTRNR2L12 MTRNR2L10 AL365361.1 RP11-343N15.5 RP11-427H3.3 AL592284.1 RP11-94L15.2 AL512274.1 DDIT4-AS1 RP11-145M9.4 RP11-611L7.1 PWAR6 RP11-690G19.3 AL078639.1 RP1-3J17.3 CTA-445C9.15 RP11-7K24.3 CTD-2521M24.9 AC009283.1 RP11-572C15.6 AC008115.3 LINC02678 RP11-439A17.7 RP11-658F2.8 MTRNR2L9 RP11-350F4.2 AC026806.2 CTD-2369P2.2 RP5-935K16.1 AP001157.1 AC024060.2 RP11-206L10.3 RP11-383J24.6 AC012510.1 RP11-277L2.3 AL590452.1 RP11-469M7.1 RP11-312O7.2 CTD-3195I5.1 RP11-258C19.7 AC004520.1 RP11-175O19.4 RP11-479G22.8 RP11-108P20.1 AC010245.2 AC018755.4 AC004257.3 RP11-160O5.1 RP11-196G18.24 RP11-395P17.11 RP11-61A14.3 RP11-665N17.4 CTB-119C2.1 RP11-288H12.4 RP11-661A12.14 RP11-4F5.2 CTC-297N7.8 RP11-458N5.1 AP005131.6 AL353804.1 RP11-391M1.4 RP11-38P22.2 AC004447.2 RP4-758J24.5 RP1-193H18.2 CTB-167B5.2 RP11-379K17.12 AC068039.4 CTC-428G20.3 RP11-309G3.3 RP11-498D10.6 C9orf38 RP11-97C16.1 RP11-277L2.4 AC018809.1 RP11-137H2.6 RP11-849F2.8 RP11-382A18.2 CTD-2587H19.1 RP11-101E13.5 RP11-187C1

In [102]:
# no way ensemble mapping is possible

In [111]:
len(set(adata_sc.var_names))

32197

In [132]:
len(set(adata_sc.var_names) & set(ensemble_csv['ensembl_gene_id_version']))

31590

In [133]:
len(set(adata_sc.var_names)) - len(set(adata_sc.var_names) & set(ensemble_csv['ensembl_gene_id_version'])) 

607

In [116]:
# 607 genes missing ensemble IDs
# add the start and end position to adata

In [134]:
dict_start = dict(list(zip(ensemble_csv['ensembl_gene_id_version'], ensemble_csv['start_position'])))
dict_end = dict(list(zip(ensemble_csv['ensembl_gene_id_version'], ensemble_csv['end_position'])))
dict_chr = dict(list(zip(ensemble_csv['ensembl_gene_id_version'], ensemble_csv['chromosome_name'])))

In [135]:
adata_sc.var['start'] = adata_sc.var_names.map(dict_start)
adata_sc.var['end'] = adata_sc.var_names.map(dict_end)
adata_sc.var['chromosome'] = adata_sc.var_names.map(dict_chr)

In [137]:
adata_sc.var['chromosome'] = 'chr' + adata_sc.var['chromosome']

In [140]:
adata_sc.var_names = adata_sc.var.gene_name

In [144]:
adata_sc.var.ensembl_id.unique().tolist()

['ENSG00000202241.1',
 'ENSG00000121410.12',
 'ENSG00000268895.6',
 'ENSG00000148584.16',
 'ENSG00000175899.15',
 'ENSG00000245105.4',
 'ENSG00000166535.20',
 'ENSG00000256661.1',
 'ENSG00000184389.9',
 'ENSG00000128274.17',
 'ENSG00000118017.4',
 'ENSG00000094914.14',
 'ENSG00000081760.17',
 'ENSG00000114771.14',
 'ENSG00000197953.6',
 'ENSG00000262466.5',
 'ENSG00000188984.12',
 'ENSG00000204518.3',
 'ENSG00000109576.14',
 'ENSG00000158122.12',
 'ENSG00000103591.13',
 'ENSG00000115977.21',
 'ENSG00000087884.16',
 'ENSG00000127837.10',
 'ENSG00000129673.10',
 'ENSG00000131043.13',
 'ENSG00000205002.4',
 'ENSG00000090861.17',
 'ENSG00000124608.5',
 'ENSG00000266967.7',
 'ENSG00000157426.14',
 'ENSG00000149313.11',
 'ENSG00000008311.16',
 'ENSG00000215458.8',
 'ENSG00000275700.6',
 'ENSG00000181409.14',
 'ENSG00000281376.1',
 'ENSG00000183044.12',
 'ENSG00000165029.17',
 'ENSG00000154263.18',
 'ENSG00000144452.15',
 'ENSG00000179869.15',
 'ENSG00000107331.18',
 'ENSG00000167972.14',
 'E

In [145]:
adata_sc.write('../single_cell_int/adata_nuc_int_outlier_genes.h5ad')

# Transfer ENSEMBLE IDs to snRNA_seq

In [17]:
adata_sc = sc.read_h5ad('../single_cell_int/adata_sc_int_outlier_genes.h5ad')
adata_sn = sc.read_h5ad('../single_nuc_int/adata_nuc_int_outlier.h5ad')

In [35]:
adata_sc.obs.Dataset.unique().tolist()

['Ding',
 'Lee',
 'Simeone',
 'Steele',
 'Peng',
 'Caronni',
 'Zhang',
 'Lin',
 'Schlesinger',
 'Steele_Adj_Norm',
 'Peng_Normal']

In [19]:
sc_var = adata_sc.var[['ensembl_id', 'start', 'end', 'chromosome']]
sc_var['var_names'] = adata_sc.var.index

In [20]:
sn_var = adata_sn.var.copy()
sn_var['var_names'] = adata_sn.var.index

In [21]:
sn_var.head()

,n_cells,n_cells_by_counts,mean_counts,log1p_mean_counts,pct_dropout_by_counts,total_counts,log1p_total_counts,var_names
7SK,3158,0,0.0,0.0,100.0,0.0,0.0,7SK
7SK-1,361,0,0.0,0.0,100.0,0.0,0.0,7SK-1
7SK-2,2295,0,0.0,0.0,100.0,0.0,0.0,7SK-2
7SK.1,40,0,0.0,0.0,100.0,0.0,0.0,7SK.1
7SK.2,193,0,0.0,0.0,100.0,0.0,0.0,7SK.2


In [22]:
sc_var.head()

,ensembl_id,start,end,chromosome,var_names
gene_name,,,,,
7SK,ENSG00000202241.1,30864250.0,30864552.0,chr6,7SK
7SK-1,ENSG00000202241.1,30864250.0,30864552.0,chr6,7SK-1
7SK-2,ENSG00000202241.1,30864250.0,30864552.0,chr6,7SK-2
7SK.1,ENSG00000202241.1,30864250.0,30864552.0,chr6,7SK.1
7SK.2,ENSG00000202241.1,30864250.0,30864552.0,chr6,7SK.2


In [23]:
merged_var = pd.merge(sn_var, sc_var, on='var_names', how='left')

In [24]:
adata_sn.var.head()

,n_cells,n_cells_by_counts,mean_counts,log1p_mean_counts,pct_dropout_by_counts,total_counts,log1p_total_counts
7SK,3158,0,0.0,0.0,100.0,0.0,0.0
7SK-1,361,0,0.0,0.0,100.0,0.0,0.0
7SK-2,2295,0,0.0,0.0,100.0,0.0,0.0
7SK.1,40,0,0.0,0.0,100.0,0.0,0.0
7SK.2,193,0,0.0,0.0,100.0,0.0,0.0


In [25]:
merged_var

,n_cells,n_cells_by_counts,mean_counts,log1p_mean_counts,pct_dropout_by_counts,total_counts,log1p_total_counts,var_names,ensembl_id,start,end,chromosome
0,3158,0,0.0,0.0,100.0,0.0,0.0,7SK,ENSG00000202241.1,30864250.0,30864552.0,chr6
1,361,0,0.0,0.0,100.0,0.0,0.0,7SK-1,ENSG00000202241.1,30864250.0,30864552.0,chr6
2,2295,0,0.0,0.0,100.0,0.0,0.0,7SK-2,ENSG00000202241.1,30864250.0,30864552.0,chr6
3,40,0,0.0,0.0,100.0,0.0,0.0,7SK.1,ENSG00000202241.1,30864250.0,30864552.0,chr6
4,193,0,0.0,0.0,100.0,0.0,0.0,7SK.2,ENSG00000202241.1,30864250.0,30864552.0,chr6
...,...,...,...,...,...,...,...,...,...,...,...,...
39019,197,0,0.0,0.0,100.0,0.0,0.0,bP-21264C1.2,ENSG00000278932.6,9325013.0,9370490.0,chr21
39020,3371,0,0.0,0.0,100.0,0.0,0.0,hsa-mir-1199,ENSG00000284081.1,14073361.0,14073479.0,chr19
39021,13,0,0.0,0.0,100.0,0.0,0.0,hsa-mir-150,ENSG00000207782.3,49500785.0,49500868.0,chr19
39022,46,0,0.0,0.0,100.0,0.0,0.0,hsa-mir-335,ENSG00000199043.1,130496111.0,130496204.0,chr7


In [26]:
adata_sn.var = merged_var.set_index('var_names')

In [32]:
adata_sn.obs.rename(columns={'Dataset_Barcode': 'Dataset_Barcode_repeated'}, inplace=True)

In [33]:
adata_sn.write('../single_nuc_int/adata_nuc_int_outlier_genes.h5ad')